### 변경점
* 변경점 : 데이터셋 변경 (캐글의 animals 데이터셋)

이 노트북에서는 CIFAR 데이터셋에서 합성곱 신경망(CNN)을 훈련하는 데 필요한 단계를 소개합니다.

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models, optimizers, utils, datasets
#from notebooks.utils import display

C:\Users\alstn\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 1. 데이터 준비 <a name="prepare"></a>
### 1) 데이터 불러오기 및 구조 확인

In [2]:
import os
import cv2
import numpy as np

# 데이터 디렉토리
data_dir = "./train"
categories = ["cat"]

# 이미지 크기
img_width, img_height = 256, 256

# 데이터를 저장할 리스트
data = []
labels = []

for category in categories:
    path = os.path.join(data_dir, category)
    class_num = 0

    for img_name in os.listdir(path)[:500]:
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        img = img / 255.0  # 정규화
        data.append(img)
        labels.append(class_num)

# 데이터와 라벨을 NumPy 배열로 변환
data_cat = np.array(data)
labels_cat = np.array(labels)

In [3]:
import os
import cv2
import numpy as np

# 데이터 디렉토리
data_dir = "./train"
categories = ["dog"]


# 데이터를 저장할 리스트
data = []
labels = []

for category in categories:
    path = os.path.join(data_dir, category)
    class_num = 1
    for img_name in os.listdir(path)[:500]:
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        img = img / 255.0  # 정규화
        data.append(img)
        labels.append(class_num)

# 데이터와 라벨을 NumPy 배열로 변환
data_dog = np.array(data)
labels_dog = np.array(labels)

In [4]:
data = np.concatenate((data_dog, data_cat))
labels = np.concatenate((labels_dog, labels_cat))

In [5]:
labels = labels.reshape(-1, 1)

### 변경점
* 변경점 : 입력데이터셋의 형태에 맞춰 inpuy_layer 변경, 스트라이드 1 고정, 패딩 X, 커널사이즈 7로 변경

## 2. 모델 만들기 <a name="build"></a>

In [38]:
input_layer = layers.Input((img_width, img_width, 3))

x = layers.Conv2D(filters=32, kernel_size=3, strides=2)(input_layer)
#x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=8, kernel_size=3, strides=2)(x)
x = layers.BatchNormalization()(x)
#x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=16, kernel_size=3, strides=2)(x)
x = layers.BatchNormalization()(x)
#x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=16, kernel_size=3, strides=2)(x)
x = layers.BatchNormalization()(x)
#x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=16, kernel_size=3, strides=2)(x)
x = layers.BatchNormalization()(x)
#x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=16, kernel_size=3, strides=2)(x)
x = layers.BatchNormalization()(x)
#x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=16, kernel_size=3)(x)
x = layers.BatchNormalization()(x)
#x = layers.LeakyReLU()(x)
#x = layers.GlobalMaxPooling2D()(x)

x = layers.Flatten()(x)
x = layers.Dense(32)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(rate=0.2)(x)

x = layers.Dense(1)(x)
output_layer = layers.Activation("softmax")(x)

model = models.Model(input_layer, output_layer)

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_74 (Conv2D)          (None, 127, 127, 32)      896       
                                                                 
 conv2d_75 (Conv2D)          (None, 63, 63, 8)         2312      
                                                                 
 batch_normalization_72 (Ba  (None, 63, 63, 8)         32        
 tchNormalization)                                               
                                                                 
 conv2d_76 (Conv2D)          (None, 31, 31, 16)        1168      
                                                                 
 batch_normalization_73 (Ba  (None, 31, 31, 16)        64        
 tchNormalization)                                        

## 3. 모델 훈련 <a name="train"></a>

In [39]:
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(
    loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"]
)

In [40]:
result = model.fit(data, labels, batch_size = 64, epochs=10, shuffle=True)

Epoch 1/10
16/16 [==============================] - 4s 116ms/step - loss: 0.8037 - accuracy: 0.5000
Epoch 2/10
16/16 [==============================] - 2s 121ms/step - loss: 0.7163 - accuracy: 0.5000
Epoch 3/10
16/16 [==============================] - 2s 116ms/step - loss: 0.6682 - accuracy: 0.5000
Epoch 4/10
16/16 [==============================] - 2s 123ms/step - loss: 0.6171 - accuracy: 0.5000
Epoch 5/10
16/16 [==============================] - 2s 118ms/step - loss: 0.5923 - accuracy: 0.5000
Epoch 6/10
16/16 [==============================] - 2s 121ms/step - loss: 0.5649 - accuracy: 0.5000
Epoch 7/10
16/16 [==============================] - 2s 117ms/step - loss: 0.5433 - accuracy: 0.5000
Epoch 8/10
16/16 [==============================] - 2s 122ms/step - loss: 0.5190 - accuracy: 0.5000
Epoch 9/10
16/16 [==============================] - 2s 131ms/step - loss: 0.4969 - accuracy: 0.5000
Epoch 10/10
16/16 [==============================] - 2s 118ms/step - loss: 0.4763 - accuracy: 0.5000

In [31]:
model.predict(data)

32/32 [==============================] - 1s 21ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

## 4. 평가 <a name="evaluate"></a>

In [ ]:
# 검증/시험 데이터셋에 평가 진행
model.evaluate(val_dataset)